***Machine Learning***

In [ ]:
# Test ML on Validation Set

from conventional.classification import model
from conventional.feature_extraction import train_features, val_features, test_features, train_classes, val_classes, test_classes
from utils import evaluate 
import matplotlib.pyplot as plt
import seaborn as sns

train_pred = model.predict(train_features)
val_pred = model.predict(val_features)
train_acc, train_f1, train_cm = evaluate(train_classes, train_pred)
val_acc, val_f1, val_cm = evaluate(val_classes, val_pred)

print(f"Training: Accuracy = {train_acc:.4f}, F1 Score = {sum(train_f1)/len(train_f1):.4f} : {train_f1}")
print(f"Validation: Accuracy = {val_acc:.4f}, F1 Score = {sum(val_f1)/len(val_f1):.4f} : {val_f1}")
plt.figure(figsize=(20, 5))
plt.subplot(1, 3, 1)
sns.heatmap(train_cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title(f'Training Confusion Matrix')
plt.subplot(1, 3, 2)
sns.heatmap(val_cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title(f'Validation Confusion Matrix')
plt.show()

In [ ]:
# Final ML Results on Testing Set

test_pred = model.predict(test_features)
test_acc, test_f1, test_cm = evaluate(test_classes, test_pred)

print(f"Testing: Accuracy = {test_acc:.4f}, F1 Score = {sum(test_f1)/len(test_f1):.4f} : {test_f1}")
plt.figure(figsize=(6, 5))
sns.heatmap(test_cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title(f'Testing Confusion Matrix')
plt.show()

***Deep Learning***

In [ ]:
# Train model with current configuration

from deep_learning.model import engine
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

results = engine.train(epochs=25, save_path = 'cp.pt')

In [ ]:
# Evaluate best checkpoint on Validation set

best_epoch = results['best_epoch']
epochs = range(len(results['train']['loss']))

print(f"\nTraining: Loss = {results['train']['loss'][best_epoch]:.4f}, Accuracy = {results['train']['accuracy'][best_epoch]:.4f}, F1 Score = {sum(results['train']['f1score'][best_epoch])/len(results['train']['f1score'][best_epoch]):.4f} : {results['train']['f1score'][best_epoch]}")
print(f"Validation: Loss = {results['val']['loss'][best_epoch]:.4f}, Accuracy = {results['val']['accuracy'][best_epoch]:.4f}, F1 Score = {sum(results['val']['f1score'][best_epoch])/len(results['val']['f1score'][best_epoch]):.4f} : {results['val']['f1score'][best_epoch]}")

plt.figure(figsize=(18, 5))
plt.subplot(1, 3, 1)
plt.plot(epochs, results['train']['loss'], label='Training Loss')
plt.plot(epochs, results['val']['loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Focal Loss')
plt.legend()
plt.subplot(1, 3, 2)
plt.plot(epochs, results['train']['accuracy'], label='Training Accuracy')
plt.plot(epochs, results['val']['accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy')
plt.legend()
plt.subplot(1, 3, 3)
plt.plot(epochs, np.mean(results['train']['f1score'], axis=-1), label='Training F1 Score')
plt.plot(epochs, np.mean(results['val']['f1score'], axis=-1), label='Validation F1 Score')
plt.xlabel('Epochs')
plt.ylabel('F1 Score')
plt.title('F1 Score')
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(18, 3))
for i in range(5):
  plt.subplot(1, 5, i+1)
  plt.plot(epochs, np.array(results['train']['f1score'])[:,i], label='Training F1 Score')
  plt.plot(epochs, np.array(results['val']['f1score'])[:,i], label='Validation F1 Score')
  plt.xlabel('Epochs')
  plt.ylabel('F1 Score')
  plt.title(f'Class {i} F1 Score')
  plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(20, 5))
best_conf_matrix = results['train']['conf_matrix'][best_epoch]
plt.subplot(1, 3, 1)
sns.heatmap(best_conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title(f'Training Confusion Matrix at Epoch {best_epoch}')
best_conf_matrix = results['val']['conf_matrix'][best_epoch]
plt.subplot(1, 3, 2)
sns.heatmap(best_conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title(f'Validation Confusion Matrix at Epoch {best_epoch}')
plt.show()

In [ ]:
# Test Final DL model on Test Set

from deep_learning.model import test_loader

engine.load('cp.pt')
test_loss, test_accuracy, test_f1, test_cm = engine.test(test_loader)
print(f"Testing: Loss = {test_loss:.4f}, Accuracy = {test_accuracy:.4f}, F1 Score = {sum(test_f1)/len(test_f1):.4f} : {test_f1}")
plt.figure(figsize=(4, 3))
sns.heatmap(test_cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Testing Confusion Matrix')
plt.show()

***Visualization***

In [ ]:
# Original vs Preprocessing 

from conventional.feature_extraction import train_loader
import matplotlib.pyplot as plt
import numpy as np

for batch in (train_loader):
    Classes = ['Healthy', 'Doubtful', 'Minimal', 'Moderate', 'Severe']
    plt.figure(figsize=(15,9))
    for i in range(len(batch['label'])):
        plt.subplot(8,8,i+1)
        plt.axis('off')
        plt.imshow(batch['image'][i][0], cmap='gray', vmin=0, vmax=255)
        plt.title(f"{Classes[batch['label'][i].item()]}")
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(15,9))
    for i in range(len(batch['label'])):
        plt.subplot(8,8,i+1)
        plt.axis('off')
        plt.imshow(batch['clahe'][i][0], cmap='gray', vmin=0, vmax=255)
        plt.title(f"{Classes[batch['label'][i].item()]}")
    plt.tight_layout()
    plt.show()
    break


In [ ]:
# AIA Steps

from conventional.feature_extraction import train_loader
import matplotlib.pyplot as plt
import numpy as np

for batch in train_loader:
    plt.figure(figsize=(15,6))
    plt.subplot(2,6,1)
    plt.axis('off')
    plt.imshow(batch['image'][0][0], cmap='gray', vmin=0, vmax=255)
    plt.title("Original Image")
    plt.subplot(2,6,2)
    plt.axis('off')
    plt.imshow(batch['not_inverted'][0][0], cmap='gray', vmin=0, vmax=255)
    plt.title("Negative Inversion")
    plt.subplot(2,6,3)
    plt.axis('off')
    plt.imshow(batch['matched'][0][0], cmap='gray', vmin=0, vmax=255)
    plt.title("Histogram Matching")
    plt.subplot(2,6,4)
    plt.axis('off')
    plt.imshow(batch['clahe'][0][0], cmap='gray', vmin=0, vmax=255)
    plt.title("CLAHE")
    plt.subplot(2,6,5)
    plt.axis('off')
    plt.imshow(batch['mean_shift'][0][0], cmap='gray', vmin=0, vmax=255)
    plt.title("Mean Shift")
    plt.subplot(2,6,6)
    plt.axis('off')
    plt.imshow(batch['not_inverted'][0][0], cmap='gray', vmin=0, vmax=255)
    plt.imshow(batch['image_thresholded'][0][0], alpha=np.where(batch['image_thresholded'][0][0] == 0, 0, 0.3))
    plt.title("Otsu Thresholding")
    plt.subplot(2,6,7)
    plt.axis('off')
    plt.imshow(batch['not_inverted'][0][0], cmap='gray', vmin=0, vmax=255)
    plt.imshow(batch['positive_marker'][0][0], alpha=np.where(batch['positive_marker'][0][0] == 0, 0, 0.3))
    plt.title("Positive Marker Extraction")
    plt.subplot(2,6,8)
    plt.axis('off')
    plt.imshow(batch['not_inverted'][0][0], cmap='gray', vmin=0, vmax=255)
    plt.imshow(batch['negative_marker'][0][0], alpha=np.where(batch['negative_marker'][0][0] == 0, 0, 0.3))
    plt.title("Negative Marker Extraction")
    plt.subplot(2,6,9)
    plt.axis('off')
    plt.imshow(batch['not_inverted'][0][0], cmap='gray', vmin=0, vmax=255)
    plt.imshow(batch['watershed'][0][0], alpha=np.where(batch['watershed'][0][0] == 0, 0, 0.3))
    plt.title("Iterative Watershed")
    plt.subplot(2,6,10)
    plt.axis('off')
    plt.imshow(batch['not_inverted'][0][0], cmap='gray', vmin=0, vmax=255)
    plt.imshow(batch['kmeans'][0][0], alpha=np.where(batch['kmeans'][0][0] == 0, 0, 0.3))
    plt.title("KMeans")
    plt.subplot(2,6,11)
    plt.axis('off')
    plt.imshow(batch['not_inverted'][0][0], cmap='gray', vmin=0, vmax=255)
    plt.imshow(batch['merged_copy'][0][0], alpha=np.where(batch['merged_copy'][0][0] == 0, 0, 0.3))
    plt.title("Watershed + Kmeans")
    plt.subplot(2,6,12)
    plt.axis('off')
    plt.imshow(batch['not_inverted'][0][0], cmap='gray', vmin=0, vmax=255)
    plt.imshow(batch['merged'][0][0], alpha=np.where(batch['merged'][0][0] == 0, 0, 0.3))
    plt.title("Holes Filling")
    plt.tight_layout()
    plt.show()


In [ ]:
# Augmentation Samples

from data.dataset import get_loader
from configs import config
from deep_learning.transforms import train_transforms
import matplotlib.pyplot as plt

bs = 3
train_loader = get_loader(config.data['path'], transforms=train_transforms, mode='train', balance=False, shuffle=False, batch_size=bs, seed=config.seed)
batch = next(iter(train_loader))
for idx in range(bs):
  plt.figure(figsize=(10,10))
  plt.subplot(5,5,1)
  plt.imshow(plt.imread(batch['image_path'][idx]), cmap='gray')
  plt.axis('off')
  plt.title('Original')
  for i in range(24):
    plt.subplot(5,5,2+i)
    for batch in train_loader:
      plt.imshow(batch['image'][idx,0].cpu(), cmap='gray')
      plt.axis('off')
      plt.title(f'Augmented {i}')
      break
  plt.show()
  print()
